In [59]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
from rnn_classes import Dog, DogInput, Race, Races, GRUNet
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
import model_predictions
from rnn_classes import smalll_lin_GRUNet, smalll_prelin_GRUNet,GRUNetv2, GRUNetv2_test,GRUNetv3
# import val_model_external
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence

In [60]:
def model_predictions(predDB:Races,model_location, net_size = 'small', new_hidden_state=None):
    torch_saved_dict = torch.load(model_location,map_location=torch.device('cpu'))
    predDB.to_cpu()

    if new_hidden_state:
        predDB.fill_hidden_states_from_dict(hidden_dict=new_hidden_state)
    else:
        predDB.fill_hidden_states_from_dict(hidden_dict=torch_saved_dict['db'])

    predDB.to_cpu()
    input_size = predDB.get_race_input([0,1])[0].full_input.shape[0]


    if net_size == 'small_lin':
        model = smalll_prelin_GRUNet(input_size,64, output='softmax')
    else:
        model = GRUNetv2(input_size,64, output='softmax')

        
        
    model.load_state_dict(torch_saved_dict['model_state_dict'])
    model.eval()
    outs = model(predDB.racesDict.values()).to('cpu')
    predDB.to_cpu()
    x = predDB.racesDict.values()
    outs_list = [item for sublist in outs.tolist() for item in sublist]
    dogs = [dog.dog.dog_name for sublist in [r.dogs for r in x] for dog in sublist]
    dog_hidden = [dog.dog.hidden_filled for sublist in [r.dogs for r in x] for dog in sublist]
    box = [r for s in [[1,2,3,4,5,6,7,8] for r in x] for r in s]
    dogId = [dog.dog.dogid for sublist in [r.dogs for r in x] for dog in sublist]
    date = [r for s in [[r.race_date]*8 for r in x] for r in s]
    times = [r for s in [[r.race_time]*8 for r in x] for r in s]
    rid = [r for s in [[r.raceid]*8 for r in x] for r in s]
    track = [r for s in [[r.track_name]*8 for r in x] for r in s]
    rnum = [r for s in [[int(r.race_num)]*8 for r in x] for r in s]
    outs = pd.DataFrame(data = {"raceid":rid, "track":track,'date':date, "racetime":times,"box":box, "dogs":dogs, "dogid":dogId,'dog_hidden':dog_hidden, "conf":outs_list,"race_num":rnum })
    outs['pred_price'] = outs['conf'].apply(lambda x: 1/(x))
    outs = outs.sort_values(['track','race_num','box'])

    return outs

In [61]:
def v6_forward_pass_train(raceDB:Races, model:GRUNetv2, basic=False):
    torch.cuda.empty_cache()
    with torch.no_grad():
        len_train_dogs = len(raceDB.train_dog_ids)
        len_train_races = len(raceDB.train_race_ids)

        dogs_idx = range(0,len_train_dogs-1)
        dogs = [inner[0].dog for inner in [x for x in raceDB.get_dog_train(dogs_idx)]] #[Dog]
        dog_input = [inner for inner in [x for x in raceDB.get_dog_train(dogs_idx)]] #[[DogInput]]


        if basic:
            train = [torch.stack(n,0) for n in [[z.stats for z in inner] for inner in [x for x in raceDB.get_dog_train(dogs_idx)]]] 
        else:
            train = [torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in [x for x in raceDB.get_dog_train(dogs_idx)]]] 
        X = pack_sequence(train, enforce_sorted=False)

        del train
        torch.cuda.empty_cache()
        #sorts dogs acroding to packed seq
        dogs_sorted = [dogs[x] for x in X[3]] #[Dog]

        output,hidden = model(X)

        for i,dog in enumerate(dog_input):
            dog_outputs = output[i]
            for j,di in enumerate(dog):
                di.hidden_out = dog_outputs[j]
    torch.cuda.empty_cache()

In [62]:
def v6_forward_pass_test(raceDB:Races, model:GRUNetv2, basic=False):
    sft_max = nn.Softmax(dim=-1)
    model.eval()
    torch.cuda.empty_cache()
    with torch.no_grad():
        len_test = len(raceDB.test_dog_ids)
        test_idx = range(0,len_test)

        dogs = [x for x in  raceDB.test_dogs.values()]  #[Dog]
        print(f"length of dogs {len(dogs)}")
        dog_input = [inner for inner in [x for x in raceDB.get_dog_test(test_idx)]] #[[DogInput]]
    
        for dog in dogs:
            dog.hidden = dog.hidden.to('cuda:0')

        if basic:
            X = pack_sequence([torch.stack(n,0) for n in [[z.stats.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)
        else:
            X = pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)
        # X = pack_sequence(train, enforce_sorted=False)

        # dogs_sorted = [dogs[x] for x in X[3]]
        # print(f"length of dogs sorted {len(dogs_sorted)}")
        hidden_in = torch.stack([x.hidden for x in dogs]).transpose(0,1)

        print(hidden_in.shape)

        # #sorts dogs acroding to packed seq
        # dogs_sorted = [dogs[x] for x in X[3]] #[Dog]

        output,_ = model(X,h=hidden_in)

        for i,dog in enumerate(dog_input):
            dog_outputs = output[i]
            # print(f'{i=}, shape of output {len(dog_outputs)} type of output {type(dog_outputs)}')
            for j,di in enumerate(dog):
                di.hidden_out = dog_outputs[j]

        raceDB.margin_from_dog_to_race_v3(mode='test')

        len_test = len(raceDB.test_race_ids)
        test_idx = range(0,len_test)

        race = raceDB.get_test_input(test_idx)

        X = torch.stack([r.hidden_in for r in race])

        output,relu = model(X, p1=False)

        relu = torch.mean(abs(relu),dim=1)

        softmax_preds = sft_max(output).to('cpu')

        test_races = raceDB.get_test_input(test_idx)

        x = race

        
        relu_out = [r for s in  [[r]*8 for r in relu.tolist()] for r in s]
        outs_list = [item for sublist in softmax_preds.tolist() for item in sublist]
        dogs = [dog.dog.dog_name for sublist in [r.dogs for r in x] for dog in sublist]
        dog_hidden = [dog.dog.hidden_filled for sublist in [r.dogs for r in x] for dog in sublist]
        dog_last_run_date = [dog.dog.last_race_date for sublist in [r.dogs for r in x] for dog in sublist]
        box = [r for s in [[1,2,3,4,5,6,7,8] for r in x] for r in s]
        dogId = [dog.dog.dogid for sublist in [r.dogs for r in x] for dog in sublist]
        date = [r for s in [[r.race_date]*8 for r in x] for r in s]
        times = [r for s in [[r.race_time]*8 for r in x] for r in s]
        rid = [r for s in [[r.raceid]*8 for r in x] for r in s]
        track = [r for s in [[r.track_name]*8 for r in x] for r in s]
        rnum = [r for s in [[int(r.race_num)]*8 for r in x] for r in s]
        outs = pd.DataFrame(data = {"raceid":rid, 
                                    "track":track,
                                    'date':date,
                                    "racetime":times,
                                    "dogid":dogId,
                                    'dog_hidden':dog_hidden,
                                    'dog_last_run_date':dog_last_run_date,
                                    'relu_sum':relu_out,
                                    "conf":outs_list,
                                    "box":box,
                                    "dogs":dogs,
                                    "race_num":rnum })
        
        outs['pred_price'] = outs['conf'].apply(lambda x: 1/(x))
        outs = outs.sort_values(['track','race_num','box'])

        
    return outs

In [63]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
#dog_stats_file = open( 'new gru input 2023-01.npy', 'rb')
CUDA_LAUNCH_BLOCKING=1
date = datetime.datetime.strptime("2023-04-08", "%Y-%m-%d").date()
today = datetime.datetime.today().strftime('%Y-%m-%d')
hidden_size = 128
simple_db = r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\gru_inputs_new_simple.npy"
pred_db = f'C:/Users/Nick/Documents/GitHub/grvmodel/Python/Database Updater/testing new outs simple {today}.npy'
# raceDB = build_dataset(simple_db, hidden_size ,state_filter=['VIC', 'NSW'])
raceDB = build_dataset(pred_db, hidden_size ,state_filter=["TAS"])

['speed_avg_1', 'speed_max_1', 'split_speed_avg_1', 'split_speed_max_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1']
(994, 21)
20
Latest date = 2023-04-21 00:00:00
(609, 23)
Latest date = 2023-04-21 00:00:00
num_features_per_dog=20


  0%|          | 0/609 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/81 [00:00<?, ?it/s]

number of races = 81, number of unique dogs = 610


c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


In [64]:
# date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
today = datetime.datetime.today().strftime('%Y-%m-%d')
raceDB.create_test_split_date(date)
raceDB.create_dogs_test_split_date()
raceDB.attach_races_to_dog_inputv2()

Train examples 0, Test examples 81


  0%|          | 0/609 [00:00<?, ?it/s]

  0%|          | 0/610 [00:00<?, ?it/s]

In [65]:
def setup_model(model_name):
    model = GRUNetv2_test(87,32,num_layers=2).to('cuda:0')
    prev_model = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
    prev_model_loaded = torch.load(prev_model,map_location=torch.device('cuda:0'))
    model.load_state_dict(prev_model_loaded['model_state_dict'])
    hidden_state_dict = prev_model_loaded['db']

    return model,hidden_state_dict

In [66]:
def setup_model_v3(model_name,model_version, input_size, hidden_size, f0_size,f1_size):
    model = GRUNetv3(input_size,hidden_size,num_layers=2, fc0_size=f0_size, fc1_size=f1_size).to('cuda:0')
    prev_model = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{model_version}.pt"
    prev_model_loaded = torch.load(prev_model,map_location=torch.device('cuda:0'))
    model.load_state_dict(prev_model_loaded['model_state_dict'])
    hidden_state_dict = prev_model_loaded['db']

    return model,hidden_state_dict

In [57]:
model_name = 'peachy-darkness-493'
model,hidden_state_dict = setup_model_v3(model_name,1600, 20,128,128,64)
raceDB.fill_hidden_states_dict_v2(hidden_state_dict)
outs = v6_forward_pass_test(raceDB, model, basic=True)
today_time = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
today = datetime.datetime.today().strftime('%Y-%m-%d')
today_date = datetime.datetime.today().date()

outs['Model Name'] = model_name
outs = outs#[ outs['track'].isin(['Richmond', 'Wentworth Park'])]
outs_1 = outs#[outs['date']==today_date]

outs_1.to_pickle(f'./model_outputs/output {model_name} {today_time}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} -{today_time}.xlsx')

  0%|          | 0/89 [00:00<?, ?it/s]

Empty: 461116469 TESTING TIME
Empty: 461116468 ELLE
Empty: 461115006 MIMI
Empty: 782171487 BIG TIME BIRDY
Empty: nullDog no_name
filled =84
empty  =5
0.9438202247191011null_dog=0
length of dogs 88
torch.Size([2, 88, 128])


In [69]:
model_name = 'graceful-jazz-516'
model,hidden_state_dict = setup_model_v3(model_name,450, 20,128,128,64)
raceDB.fill_hidden_states_dict_v2(hidden_state_dict)
outs = v6_forward_pass_test(raceDB, model, basic=True)
today_time = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
today = datetime.datetime.today().strftime('%Y-%m-%d')
today_date = datetime.datetime.today().date()

outs['Model Name'] = model_name
outs = outs#[ outs['track'].isin(['Richmond', 'Wentworth Park'])]
outs_1 = outs#[outs['date']==today_date]

outs_1.to_pickle(f'./model_outputs/output {model_name} {today_time}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} -{today_time}.xlsx')

  0%|          | 0/610 [00:00<?, ?it/s]

Empty: 771724529 RITZA LEWIS
Empty: 775589228 ROMAN WOLF
Empty: 669959782 NEW HIGHS
Empty: 547603534 KOJONUP LASS
Empty: 734719590 GAIL UNLEASHED
Empty: 566431612 TREACHEROUS
Empty: 650078173 AYLA
Empty: 632134756 ASTON PROSPECT
Empty: 615180911 ASTON HOGAN
Empty: 632134757 ASTON DOUGIE
Empty: 707215137 EMOLGA BALE
Empty: 705700606 COSMIC OVERSEER
Empty: 734718583 ZIPPING SIRI
Empty: 701016751 XATU BALE
Empty: 701016759 ROCKET BALE
Empty: 723093304 CRIMSON ROSETTE
Empty: 707215139 CATERPIE BALE
Empty: 734718585 ZIPPING ALEXA
Empty: 771741592 REMEMBER WHAT
Empty: 714641938 SWOOPING SWAN
Empty: 655358100 MANILA FLASH
Empty: 706349883 MISSIN THE BUG
Empty: nullDog no_name
filled =587
empty  =23
0.9622950819672131null_dog=0
length of dogs 609
torch.Size([2, 609, 128])


In [36]:
model_name = 'valiant-sky-456'
model,hidden_state_dict = setup_model(model_name)
raceDB.fill_hidden_states_dict_v2(hidden_state_dict)
outs = v6_forward_pass_test(raceDB, model)
today_time = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
today = datetime.datetime.today().strftime('%Y-%m-%d')
today_date = datetime.datetime.today().date()

outs['Model Name'] = model_name
outs = outs# [outs['track'].isin(['Gosford', 'Horsham' , 'Bulli', 'Casino'])]
outs_1 = outs#[outs['date']==today_dat

# outs_1 = outs[outs['pred_price'].isin([5.705377513,7.121924725,8.389350458,8.584420235,9.4191019,9.510855009,8.889639176,8.011170227,])]

outs_1.to_pickle(f'./model_outputs/output {model_name} {today_time}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} -{today_time}.xlsx')

  0%|          | 0/272 [00:00<?, ?it/s]

Empty: 707214698 CHECK THE RADAR
Empty: 707215124 KIARNI GAZELLE
Empty: 714917159 BLAST CAFE
Empty: 714917640 LESS IS MORE
Empty: 714917177 LIKE WILDFIRE
Empty: 747808187 DIG DEEP INDY
Empty: 707215201 DECENT GUY
Empty: 726169486 MISS MCGRAW
Empty: 707215210 MARVEL MALVIN
Empty: 707215211 MAILMAN MALONE
Empty: 663396149 SWEET APEX
Empty: 672069459 SLINGSHOT CONNOR
Empty: 752069638 PROMPT RESPONSE
Empty: nullDog no_name
filled =258
empty  =14
0.9485294117647058null_dog=0
length of dogs 271
torch.Size([2, 271, 32])


In [11]:
model_name = 'valiant-sky-456'
prev_model = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
prev_model_loaded = torch.load(prev_model,map_location=torch.device('cpu'))

In [12]:
hidden_state_dict = prev_model_loaded['db']

In [13]:
afire = hidden_state_dict['678122538'][0]
print(afire)

tensor([[ 1.0000e+00, -1.0000e+00,  0.0000e+00,  9.6269e-01, -1.0000e+00,
          1.0000e+00,  9.1328e-01,  1.0000e+00,  0.0000e+00, -1.0000e+00,
         -2.1083e-01,  0.0000e+00, -9.9660e-01,  1.0000e+00, -7.1763e-05,
          1.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+00,  1.0000e+00,
          9.8803e-01, -5.4968e-01,  0.0000e+00, -9.8572e-01, -1.0000e+00,
          1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00, -9.3613e-01,
         -1.0000e+00, -1.0000e+00],
        [ 9.1053e-01, -2.7656e-01,  1.0000e+00, -9.4494e-02,  9.6896e-01,
          1.0000e+00,  9.9049e-01,  1.0000e+00, -9.0974e-02,  1.0000e+00,
          9.9997e-01,  1.0000e+00,  1.0000e+00,  8.2621e-01,  1.0000e+00,
         -1.0000e+00,  9.7446e-01,  1.0000e+00,  9.9999e-01,  9.9986e-01,
          9.9999e-01,  9.7507e-01, -9.8294e-01,  1.0000e+00, -1.0000e+00,
          9.4714e-01,  1.0000e+00,  1.0000e+00, -3.7325e-01, -8.6453e-01,
         -1.0000e+00,  8.9387e-01]])


In [ ]:
kingsley = hidden_state_dict['665757358'][0].to('cpu')
print(kingsley)

In [30]:
r1 = raceDB.racesDict['889002573']
[d.hidden_out for d in r1.dogs]

[tensor([-0.0238, -0.1045,  1.0000, -0.3999,  0.0022,  0.9964, -0.0448,  1.0000,
          0.0081,  1.0000,  0.9862,  1.0000, -0.0897, -0.6760,  1.0000, -1.0000,
         -0.4448,  1.0000,  1.0000,  1.0000,  1.0000, -0.0140, -0.0490,  1.0000,
         -0.9885,  0.0660,  1.0000,  0.9948,  0.1450,  0.0219, -1.0000, -0.0561],
        device='cuda:0'),
 tensor([ 4.8473e-01, -2.8674e-01,  1.0000e+00, -2.9826e-01,  6.2193e-01,
          1.0000e+00,  7.0683e-01,  1.0000e+00, -8.8372e-04,  1.0000e+00,
          9.9984e-01,  1.0000e+00,  9.9912e-01,  3.3114e-01,  1.0000e+00,
         -1.0000e+00,  8.4187e-01,  1.0000e+00,  1.0000e+00,  9.9999e-01,
          1.0000e+00,  3.5037e-01, -8.3341e-01,  1.0000e+00, -9.9998e-01,
          6.8520e-01,  1.0000e+00,  9.9997e-01, -3.2998e-01, -7.7432e-01,
         -1.0000e+00,  4.1243e-01], device='cuda:0'),
 tensor([ 0.0588, -0.3857,  1.0000, -0.5911,  0.1186,  0.8024,  0.1212,  1.0000,
          0.2140,  1.0000,  0.9985,  1.0000,  0.0520, -0.4569,  1.0000

In [ ]:
r2 = raceDB.racesDict['888958937']
[d.hidden_out for d in r2.dogs]

In [12]:
r2 = raceDB.racesDict['888958937']
[d.hidden_out for d in r2.dogs]

[tensor([ 0.9106, -0.2766,  1.0000, -0.0949,  0.9689,  1.0000,  0.9905,  1.0000,
         -0.0905,  1.0000,  1.0000,  1.0000,  1.0000,  0.8262,  1.0000, -1.0000,
          0.9744,  1.0000,  1.0000,  0.9999,  1.0000,  0.9750, -0.9829,  1.0000,
         -1.0000,  0.9473,  1.0000,  1.0000, -0.3731, -0.8646, -1.0000,  0.8934],
        device='cuda:0'),
 tensor([ 0.8620, -0.3385,  1.0000, -0.2561,  0.9210,  1.0000,  0.9750,  1.0000,
         -0.0543,  1.0000,  1.0000,  1.0000,  1.0000,  0.8020,  1.0000, -1.0000,
          0.9561,  1.0000,  1.0000,  0.9999,  1.0000,  0.9059, -0.9992,  1.0000,
         -1.0000,  0.9096,  1.0000,  1.0000, -0.2822, -0.8409, -1.0000,  0.7926],
        device='cuda:0'),
 tensor([ 0.5171, -0.3337,  1.0000, -0.1794,  0.6409,  1.0000,  0.6957,  1.0000,
          0.0075,  1.0000,  0.9998,  1.0000,  1.0000,  0.4504,  1.0000, -1.0000,
          0.8260,  1.0000,  1.0000,  1.0000,  1.0000,  0.3620, -0.9148,  1.0000,
         -1.0000,  0.7174,  1.0000,  1.0000, -0.3489, -

In [13]:
r5 = raceDB.racesDict['889002554']
[d.hidden_out for d in r5.dogs]

[tensor([ 0.8786, -0.3437,  1.0000, -0.2264,  0.9437,  1.0000,  0.9808,  1.0000,
         -0.0790,  1.0000,  1.0000,  1.0000,  1.0000,  0.8052,  1.0000, -1.0000,
          0.9682,  1.0000,  1.0000,  0.9999,  1.0000,  0.9474, -0.9994,  1.0000,
         -1.0000,  0.9246,  1.0000,  1.0000, -0.3085, -0.8532, -1.0000,  0.8532],
        device='cuda:0'),
 tensor([ 0.9456, -0.3510,  1.0000, -0.1925,  0.9886,  1.0000,  0.9953,  1.0000,
         -0.1668,  1.0000,  1.0000,  1.0000,  1.0000,  0.8488,  1.0000, -1.0000,
          0.9806,  1.0000,  1.0000,  0.9998,  1.0000,  0.9844, -0.9989,  1.0000,
         -1.0000,  0.9610,  1.0000,  1.0000, -0.3682, -0.8771, -1.0000,  0.9340],
        device='cuda:0'),
 tensor([ 0.5399,  0.3814,  1.0000, -0.4527,  0.7558,  1.0000,  0.8464,  1.0000,
         -0.2268,  0.1155,  0.9999,  1.0000,  0.9999,  0.4774,  1.0000, -1.0000,
          0.8484,  1.0000,  1.0000,  1.0000,  1.0000,  0.4048, -0.3867,  1.0000,
         -0.9999,  0.6606,  1.0000,  0.9998,  0.0139, -

In [18]:
X

tensor([[ 5.1500e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.

In [31]:
X = torch.stack([r.hidden_in for r in [r1,r2,r5]])
model(X, p1=False)

tensor([[-0.7482,  2.0313, -0.8095,  0.1577, -1.3425, -2.5996, -0.4175,  2.8692],
        [ 0.2941,  0.0723, -0.0915, -0.1145, -0.2073, -0.2170, -0.1494, -0.0454],
        [ 0.2941,  0.0723, -0.0915, -0.1145, -0.2073, -0.2170, -0.1494, -0.0454]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [32]:
rl1 = model.rl1(X.float())
fc1 = model.fc1(rl1)
rl2 = model.rl2(fc1)
fc2 = model.fc2(rl2)

In [34]:
fc1

tensor([[  2.6254,   1.7583,   0.1411,  -3.2357,  -4.0776,   3.2286,  -4.7894,
         -16.6218,  -9.1629,  -6.7510, -15.4404,  -1.4580,   1.5123,   1.2001,
           2.7232,  -8.0677, -25.2226, -10.2728,  -4.2995,  -3.2322, -11.8672,
           4.7934, -17.5518, -13.5397,   1.6137,   0.5465, -29.5264,   3.5766,
          -8.8270,   0.3942,  -0.2158, -24.7863, -11.7491, -11.2108,   0.9078,
          -4.2446,   2.2536, -13.1783, -20.6844,  -2.3124,  -3.0117,   2.7220,
           2.1446, -19.4526, -18.0167,  -4.6072,   3.5481,   6.2097,   1.6527,
         -11.8739, -28.4224,  -4.3799,  -2.8509, -11.1404,  -7.7076, -13.7618,
          -5.6789,  -1.2436, -14.1827,  -2.7575,  -7.0737,  -6.3545, -11.5823,
         -16.0918],
        [ -5.0322,  -2.4691,  -6.1479, -19.0814,  -5.2419,  -4.2905,  -1.6341,
         -12.2911, -17.9578, -17.8259, -20.6278, -18.0657, -18.9996,  -5.9878,
          -6.4022, -14.6446, -28.6712, -17.6169,  -9.4557,  -5.6060, -23.7683,
          -5.7024, -27.2382, -19

In [33]:
rl2

tensor([[2.6254, 1.7583, 0.1411, 0.0000, 0.0000, 3.2286, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.5123, 1.2001, 2.7232, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 4.7934, 0.0000, 0.0000, 1.6137, 0.5465, 0.0000,
         3.5766, 0.0000, 0.3942, 0.0000, 0.0000, 0.0000, 0.0000, 0.9078, 0.0000,
         2.2536, 0.0000, 0.0000, 0.0000, 0.0000, 2.7220, 2.1446, 0.0000, 0.0000,
         0.0000, 3.5481, 6.2097, 1.6527, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0

In [28]:
fc2

tensor([[ 0.2941,  0.0723, -0.0915, -0.1145, -0.2073, -0.2170, -0.1494, -0.0454],
        [ 0.2941,  0.0723, -0.0915, -0.1145, -0.2073, -0.2170, -0.1494, -0.0454]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [21]:
X

tensor([[ 5.1500e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.

In [10]:
afire = hidden_state_dict['678122538'][0].to('cpu')
kingsley = hidden_state_dict['665757358'][0].to('cpu')

In [ ]:
afire

In [9]:
raceDB.fill_hidden_states_dict_v2(hidden_state_dict)

  0%|          | 0/375 [00:00<?, ?it/s]

Empty: 699830875 HILLBILLY LEXI
Empty: 694775838 MIGHTY STING
Empty: 752070205 KAIZEN SHADOW
Empty: 710478288 TINY BUTTON
Empty: 710477775 RIVER JOHN
Empty: 698472213 VELOCITY KIANA
Empty: 645795602 WE BELONG
Empty: 573719163 IN MEMORIAM PEST
Empty: 726168452 SUNDAY SLEEP IN
Empty: 726168453 LAVENDER HAZE
Empty: 726169451 MY PIRATE
Empty: 620126279 DRIVE THROUGH
Empty: 611215541 CAWBOURNE RUSTY
Empty: 637973443 ANITA KEROMA
Empty: 736205973 SPOTTY CAN RUN
Empty: 660657682 MATHOURA MARVEL
Empty: 717092823 OBSESSIVE QUEEN
Empty: 675307030 IASON BALE
Empty: 745110246 RUNNING ROMAN
Empty: 722114856 SUDDEN HURRICANE
Empty: 717092827 MY GIRL PIPER
Empty: 766471884 CABAO COCO
Empty: 766471886 CABAO ELDERBERRY
Empty: 766471885 CABAO DEWBERRY
Empty: 708919009 DANIEL
Empty: 745076699 BINNY DOO
Empty: nullDog no_name
filled =348
empty  =27
0.928null_dog=0


In [10]:
raceDB.dogsDict['638802667'].hidden_filled

KeyError: '638802667'

In [11]:
outs = v6_forward_pass_test(raceDB, model)
today_time = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
today = datetime.datetime.today().strftime('%Y-%m-%d')
today_date = datetime.datetime.today().date()

outs['Model Name'] = model_name
outs = outs#[ outs['track'].isin(['Richmond', 'Wentworth Park'])]
outs_1 = outs#[outs['date']==today_date]

outs_1.to_pickle(f'./model_outputs/output {model_name} {today_time}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} -{today_time}.xlsx')

length of dogs 374
torch.Size([2, 374, 32])


In [11]:
model_name = 'volcanic-wood-404'
model = setup_model(model_name)
v6_forward_pass_train(raceDB, model)
outs = v6_forward_pass_test(raceDB, model)
today_time = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
today = datetime.datetime.today().strftime('%Y-%m-%d')
today_date = datetime.datetime.today().date()

outs['Model Name'] = model_name

outs_1 = outs[outs['date']==today_date]

outs_1.to_pickle(f'./model_outputs/output {model_name} {today_time}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} -{today_time}.xlsx')

TypeError: itemgetter expected 1 argument, got 0

In [16]:
model_name = 'still-disco-178'
model = setup_model(model_name)
v6_forward_pass_train(raceDB, model)
outs = v6_forward_pass_test(raceDB, model)
today_time = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
today = datetime.datetime.today().strftime('%Y-%m-%d')
today_date = datetime.datetime.today().date()

outs['Model Name'] = model_name

outs_1 = outs[outs['date']==today_date]

outs_1.to_pickle(f'./model_outputs/output {model_name} {today_time}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} -{today_time}.xlsx')

torch.Size([1, 6071, 64])


In [14]:
today_date = datetime.datetime.today().date()
outs[outs['date']==today_date]

,raceid,track,date,racetime,box,dogs,dogid,conf,race_num,pred_price
67768,880939579,Grafton,2023-03-19,2023-03-19 18:16:00,1,KRATOS RISING,637972964,0.173082,1,5.777605
67769,880939579,Grafton,2023-03-19,2023-03-19 18:16:00,2,JUST MUMMULGUM,643634738,0.211303,1,4.732538
67770,880939579,Grafton,2023-03-19,2023-03-19 18:16:00,3,no_name,nullDog,0.020072,1,49.820692
67771,880939579,Grafton,2023-03-19,2023-03-19 18:16:00,4,ICE COOL KAOS,710478167,0.160101,1,6.246064
67772,880939579,Grafton,2023-03-19,2023-03-19 18:16:00,5,JACOB VALLEY,580792883,0.089600,1,11.160709
...,...,...,...,...,...,...,...,...,...,...
67763,880939556,Richmond,2023-03-19,2023-03-19 21:57:00,4,SANTANA FLY,707215218,0.145652,12,6.865693
67764,880939556,Richmond,2023-03-19,2023-03-19 21:57:00,5,no_name,nullDog,0.006609,12,151.300745
67765,880939556,Richmond,2023-03-19,2023-03-19 21:57:00,6,PALERMO STING,592711158,0.129751,12,7.707042
67766,880939556,Richmond,2023-03-19,2023-03-19 21:57:00,7,CHLOE CHAOS,740359826,0.031333,12,31.915401


In [9]:
model_name = 'hardy-pond-174'
model = setup_model(model_name)
v6_forward_pass_train(raceDB, model)
outs = v6_forward_pass_test(raceDB, model)
today_time = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
today = datetime.datetime.today().strftime('%Y-%m-%d')
today_date = datetime.datetime.today().date()

outs['Model Name'] = model_name

outs_1 = outs[outs['date']==today_date]

outs_1.to_pickle(f'./model_outputs/output {model_name} {today_time}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} -{today_time}.xlsx')

torch.Size([1, 5812, 64])


In [ ]:
today_date = datetime.datetime.today()
outs_1 = outs[outs['date']==today_date]

In [ ]:
outs[outs['race_num']!=-1]

,raceid,track,date,racetime,box,dogs,dogid,conf,race_num,pred_price
75288,882498477,Healesville,2023-03-19,2023-03-19 11:06:00,1,ASTON JERRY,648012438,0.105658,1,9.464539
75289,882498477,Healesville,2023-03-19,2023-03-19 11:06:00,2,SIR MELVILLE,677030363,0.108754,1,9.195083
75290,882498477,Healesville,2023-03-19,2023-03-19 11:06:00,3,ACCOUNTED FOR,644728395,0.078954,1,12.665541
75291,882498477,Healesville,2023-03-19,2023-03-19 11:06:00,4,GEOFF'S STORIES,695938098,0.246108,1,4.063255
75292,882498477,Healesville,2023-03-19,2023-03-19 11:06:00,5,KEEPS ON,659900668,0.113846,1,8.783809
...,...,...,...,...,...,...,...,...,...,...
75195,882478938,Warragul,2023-03-19,2023-03-19 21:54:00,4,ELENOR BALE,634935343,0.146202,12,6.839869
75196,882478938,Warragul,2023-03-19,2023-03-19 21:54:00,5,LADY WENDY HOOK,570556339,0.124175,12,8.053171
75197,882478938,Warragul,2023-03-19,2023-03-19 21:54:00,6,IMPRESS PATCH,465423055,0.158166,12,6.322465
75198,882478938,Warragul,2023-03-19,2023-03-19 21:54:00,7,LIST OF SECRETS,589547246,0.174829,12,5.719865


In [ ]:
outs['date'][0]

656      2022-08-03
664      2022-08-03
672      2022-08-03
680      2022-08-03
688      2022-08-03
            ...    
74951    2023-03-16
74959    2023-03-16
74967    2023-03-16
74975    2023-03-16
74983    2023-03-16
Name: date, Length: 75448, dtype: object

In [ ]:
outs = outs[outs['race_num']!=-1]

In [ ]:
outs.to_excel('test_outs_v6.xlsx')

In [ ]:
dogs = [x for x in  raceDB.train_dogs.values()]
len(dogs)

12689

In [ ]:
dogs = [x for x in  raceDB.test_dogs.values()]
len(dogs)

5792

In [ ]:
def update_hidden_state(raceDB:Races,model_name):
    nsw_model_loaded = torch.load(model_name,map_location=torch.device('cuda:0'))
    raceDB.fill_hidden_states_from_dict(hidden_dict=nsw_model_loaded['db'])
    input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0]
    print(input_size)
    raceDB.to_cuda()
    nsw_model_full = smalll_prelin_GRUNet(input_size=input_size,hidden_size=64).to('cuda:0')
    nsw_model_full.load_state_dict(nsw_model_loaded['model_state_dict'])

    races = raceDB.get_test_input(range(0,len(raceDB.test_race_ids)))

    nsw_model_full.eval()
    with torch.no_grad():
        last = 0
        for i in trange(500,len(raceDB.test_race_ids)-500,500):
            sub_races = races[last:i]
            try:
                nsw_model_full(sub_races)
            except:
                print('failed')
            last = i

        outs_full = nsw_model_full(races)
    raceDB.to_cuda()
    raceDB.create_hidden_states_dict()
    
    new_rdb = raceDB.hidden_states_dict_gru
    return new_rdb

In [ ]:
def gen_predictions(model_name, raceDB, predDB):
    model = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
    hidden_states = update_hidden_state(raceDB, model)
    outs = model_predictions(predDB, model, net_size='small_lin', new_hidden_state=hidden_states )
    return outs

In [ ]:
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
today = datetime.datetime.today().strftime('%Y-%m-%d')
pred_df = pd.read_pickle(f'C:/Users/Nick/Documents/GitHub/grvmodel/Python/Database Updater/testing new outs {today}.npy')
pred_df
state = ['NSW']
pred_df_nsw = pred_df[pred_df['state'].isin(state)]
hidden_size = 64
predDB_nsw = build_pred_dataset(pred_df_nsw, hidden_size)

77


  0%|          | 0/212 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/30 [00:00<?, ?it/s]

number of races = 30, number of unique dogs = 212


c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


In [ ]:
# model_name = "zany-wood-268"
# torch_saved_dict = torch.load(f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt")
# input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0]
# model = smalll_prelin_GRUNet(input_size=input_size,hidden_size=64).to('cuda:0')
# model.load_state_dict(torch_saved_dict['model_state_dict'])
# importlib.reload(val_model_external)
# val_model_external.validate_model(model,raceDB,model_name)

In [ ]:
model_name = "neat-universe-4"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
outs['Model Name'] = model_name

today = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
outs.replace('Richmond Straight','Richmond',inplace = True)
outs.replace('Richmond Straight','Richmond',inplace = True)

outs_1 = outs[outs['track'].isin(['Dubbo', 'Wentworth Park'])]
outs_1.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} {state}-{today}.xlsx')

  0%|          | 0/40877 [00:00<?, ?it/s]

filled =2451
empty  =324565
0.007495046113951611null_dog=0
686


100%|██████████| 15/15 [00:00<00:00, 28.59it/s]


  0%|          | 0/30 [00:00<?, ?it/s]

filled =205
empty  =35
0.8541666666666666null_dog=0


In [ ]:
model_name = "polar-paper-2"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
outs['Model Name'] = model_name

today = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
#outs.replace('Richmond Straight','Richmond',inplace = True)

outs_1 = outs[outs['track'].isin(['Dubbo', 'Wentworth Park'])]
outs_1.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} {state}-{today}.xlsx')

  0%|          | 0/40877 [00:00<?, ?it/s]

filled =272304
empty  =54712
0.8326932015558872null_dog=0
686


100%|██████████| 15/15 [00:00<00:00, 46.69it/s]


  0%|          | 0/30 [00:00<?, ?it/s]

filled =205
empty  =35
0.8541666666666666null_dog=0


: 

In [ ]:
model_name = "fallen-planet-281"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
outs['Model Name'] = model_name

today = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
#outs.replace('Richmond Straight','Richmond',inplace = True)

outs_1 = outs[outs['track'].isin(['Richmond', 'Wentworth Park'])]
outs_1.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} {state}-{today}.xlsx')
outs_carress = outs

100%|██████████| 89344/89344 [00:19<00:00, 4467.45it/s]


filled =272321
empty  =442431
0.381000682754298null_dog=0
510


100%|██████████| 47/47 [00:00<00:00, 2763.08it/s]


filled =321
empty  =55
0.8537234042553191null_dog=0


In [ ]:
model_name = "true-grass-258"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
outs['Model Name'] = model_name

today = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
outs.replace('Richmond Straight','Richmond',inplace = True)
outs_1 = outs[outs['track'].isin(['Healesville', 'Taree', 'Richmond', 'Wentworth Park'])]
outs_1.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs_1.to_excel(f'./model_outputs/output {model_name} {state}-{today}.xlsx')
outs_carress = outs

100%|██████████| 89344/89344 [00:33<00:00, 2695.36it/s]


filled =605112
empty  =109640
0.8466041368194842null_dog=0
510


100%|██████████| 80/80 [00:00<00:00, 2351.35it/s]


filled =541
empty  =99
0.8453125null_dog=0


In [ ]:
model_name = "vibrant-energy-237"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
outs['Model Name'] = model_name
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.replace('Richmond Straight','Richmond',inplace = True)
today = datetime.datetime.today().strftime('%Y-%m-%d %H_%M_%S')
outs_2= outs[outs['track'].isin(['Dubbo','Wentworth Park'])]
outs_2.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs_2.to_csv(f'./model_outputs/output {model_name} {state}-{today}.csv')
outs_carress = outs

100%|██████████| 89184/89184 [00:16<00:00, 5414.67it/s] 


filled =274383
empty  =439089
0.38457430705059203null_dog=0
510


100%|██████████| 57/57 [00:00<00:00, 2372.91it/s]


filled =411
empty  =45
0.9013157894736842null_dog=0


In [ ]:
outs_combined = pd.concat([outs_1,outs_2])
outs_combined.to_pickle(f'./model_outputs/output {model_name} combined {today}.npy')
outs_combined.to_csv(f'./model_outputs/output combined {model_name} {state}-{today}.csv')

In [ ]:
model_name = "upbeat-plant-202"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
outs['Model Name'] = model_name
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.replace('Richmond Straight','Richmond',inplace = True)
outs = outs[outs['track'].isin(['Richmond'])]
outs.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs.to_csv(f'./model_outputs/output {model_name} {state}-{today}.csv')
outs_carress = outs

100%|██████████| 162850/162850 [00:30<00:00, 5360.34it/s]


filled =275245
empty  =1027555
0.21127187595947192null_dog=0
510


100%|██████████| 24/24 [00:00<00:00, 2397.89it/s]


filled =172
empty  =20
0.8958333333333334null_dog=0


In [ ]:
model_name = "fearless-shadow-200"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
outs['Model Name'] = model_name
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.replace('Richmond Straight','Richmond',inplace = True)
outs.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs.to_csv(f'./model_outputs/output {model_name} {state}-{today}.csv')
outs_carress = outs

100%|██████████| 161848/161848 [00:21<00:00, 7458.84it/s]


filled =273708
empty  =1021076
0.21139278829519054null_dog=0
510


100%|██████████| 42/42 [00:00<00:00, 2797.49it/s]


filled =293
empty  =43
0.8720238095238095null_dog=0


In [ ]:
model_name = "effortless-sweep-1"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
outs['Model Name'] = model_name
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs.to_csv(f'./model_outputs/output {model_name} {state}-{today}.csv')
outs_carress = outs

100%|██████████| 161871/161871 [00:20<00:00, 7836.88it/s]


filled =273509
empty  =1021459
0.21120908006993222null_dog=0
512


100%|██████████| 18/18 [00:00<00:00, 2441.15it/s]


filled =126
empty  =18
0.875null_dog=0


In [ ]:
model_name = "dulcet-waterfall-189"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
outs['Model Name'] = model_name
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs.to_csv(f'./model_outputs/output {model_name} {state}-{today}.csv')
outs_carress = outs

100%|██████████| 161871/161871 [00:20<00:00, 7843.07it/s]


filled =273509
empty  =1021459
0.21120908006993222null_dog=0
512


100%|██████████| 31/31 [00:00<00:00, 2581.01it/s]


filled =223
empty  =25
0.8991935483870968null_dog=0


In [ ]:

model_name = "forthright-caress-156"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
outs['Model Name'] = model_name
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs.to_csv(f'./model_outputs/output {model_name} {state}-{today}.csv')
outs_carress = outs

100%|██████████| 161871/161871 [00:19<00:00, 8152.61it/s]


filled =273415
empty  =1021553
0.2111364914036486null_dog=0
512


RuntimeError: Error(s) in loading state_dict for smalll_lin_GRUNet:
	size mismatch for batchnorm.weight: copying a param with shape torch.Size([416]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for batchnorm.bias: copying a param with shape torch.Size([416]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for batchnorm.running_mean: copying a param with shape torch.Size([416]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for batchnorm.running_var: copying a param with shape torch.Size([416]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for fc0.weight: copying a param with shape torch.Size([416, 416]) from checkpoint, the shape in current model is torch.Size([512, 512]).
	size mismatch for fc0.bias: copying a param with shape torch.Size([416]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for gru1.weight_ih: copy

In [ ]:
model_name = "stilted-sweep-15"
outs = gen_predictions("stilted-sweep-15", raceDB, predDB_nsw)
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs.to_csv(f'./model_outputs/output {model_name} {state}-{today}.csv')
outs_stilted = outs

100%|██████████| 161170/161170 [00:22<00:00, 7209.10it/s]


filled =271623
empty  =1017737
0.2106649810758826null_dog=0


100%|██████████| 34/34 [00:00<00:00, 2426.43it/s]


filled =251
empty  =21
0.9227941176470589null_dog=0


In [ ]:
model_name = "visionary-sweep-1"
outs = gen_predictions(model_name, raceDB, predDB_nsw)
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs.to_csv(f'./model_outputs/output {model_name} {state}-{today}.csv')
outs_vision = outs

100%|██████████| 161170/161170 [00:23<00:00, 6890.25it/s]


filled =271623
empty  =1017737
0.2106649810758826null_dog=0


100%|██████████| 34/34 [00:00<00:00, 2264.49it/s]


filled =251
empty  =21
0.9227941176470589null_dog=0


In [ ]:
model_name = 'forthright-quiver-153'
outs = gen_predictions(model_name, raceDB, predDB_nsw)
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs.to_csv(f'./model_outputs/output {model_name} {state}-{today}.csv')
outs_quiver = outs

100%|██████████| 161170/161170 [00:22<00:00, 7077.21it/s]


filled =272214
empty  =1017146
0.21112334801762114null_dog=0


100%|██████████| 34/34 [00:00<00:00, 1698.54it/s]


filled =252
empty  =20
0.9264705882352942null_dog=0


In [ ]:
model_name = 'prime-sweep-8'
outs = gen_predictions(model_name, raceDB, predDB_nsw)
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.to_pickle(f'./model_outputs/output {model_name} {today}.npy')
outs.to_csv(f'./model_outputs/output {model_name} {state}-{today}.csv')
outs_prime = outs

100%|██████████| 161170/161170 [00:22<00:00, 7161.66it/s]


filled =272214
empty  =1017146
0.21112334801762114null_dog=0


100%|██████████| 34/34 [00:00<00:00, 2264.17it/s]


filled =251
empty  =21
0.9227941176470589null_dog=0


In [ ]:
all_preds = outs[['raceid','track','racetime','box','dogs','dogid']]
all_preds_merge = all_preds.merge(outs_stilted, on=['raceid','box'])

NameError: name 'outs_carress' is not defined

In [ ]:
outs = gen_predictions("pleasant-sweep-3", raceDB, predDB_nsw)

In [ ]:

nsw_model =  f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
hidden_states = update_hidden_state(raceDB, nsw_model)

  0%|          | 0/39546 [00:00<?, ?it/s]

filled =267422
empty  =48946
0.8452877661457543null_dog=0


In [ ]:
model_name = "pleasant-sweep-3"
nsw_model =  f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
hidden_states_pleasent = update_hidden_state(raceDB, nsw_model)

  0%|          | 0/39546 [00:00<?, ?it/s]

filled =267422
empty  =48946
0.8452877661457543null_dog=0


In [ ]:
outs_nsw = model_predictions(predDB_nsw, nsw_model, net_size='small_lin', new_hidden_state=hidden_states )
outs = outs_nsw

  0%|          | 0/34 [00:00<?, ?it/s]

filled =247
empty  =25
0.9080882352941176null_dog=0


In [ ]:
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.to_pickle(f'output {model_name} {today}.npy')
outs.to_csv(f'output {model_name} {state}-{today}.csv')

In [ ]:
nsw_model_loaded = torch.load(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\savedmodel\pleasant-sweep-3\pleasant-sweep-3_450.pt",map_location=torch.device('cuda:0'))
raceDB.fill_hidden_states_from_dict(hidden_dict=nsw_model_loaded['db'])
input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0]
raceDB.to_cuda()
nsw_model_full = smalll_lin_GRUNet(input_size=input_size,hidden_size=64).to('cuda:0')
nsw_model_full.load_state_dict(nsw_model_loaded['model_state_dict'])


  0%|          | 0/39546 [00:00<?, ?it/s]

filled =267422
empty  =48946
0.8452877661457543null_dog=0


<All keys matched successfully>

In [ ]:
races = raceDB.get_test_input(range(0,2542))

In [ ]:
nsw_model_full.eval()
with torch.no_grad():
    outs_full = nsw_model_full(races)

In [ ]:
raceDB.create_hidden_states_dict()
new_rdb = raceDB.hidden_states_dict


  0%|          | 0/34 [00:00<?, ?it/s]

filled =241
empty  =31
0.8860294117647058null_dog=0


In [ ]:
state = "NZ"
pred_df_nz = pred_df[pred_df['state'].str.contains(state, na=False)]
hidden_size = 64
predDB_nz = build_pred_dataset(pred_df_nz, hidden_size)

  0%|          | 0/175 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/24 [00:00<?, ?it/s]

number of races = 24, number of unique dogs = 175


c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


In [ ]:
(pred_df_nsw['prev_race']==-1).sum()

11

In [ ]:
nsw_model =  r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\savedmodel\pleasant-sweep-3\pleasant-sweep-3_450.pt"
nz_model = r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\savedmodel\very long run RUN NZ\very long run RUN NZ_450.pt"

In [ ]:
all_data = r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\DATA\new_windows_gru_REAL.npy'

In [ ]:
importlib.reload(model_predictions)

<module 'model_predictions' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\model_predictions.py'>

In [ ]:
test_R = predDB_nsw.get_race_input([1,2])[0]

In [ ]:
len(test_R.race_track)

71

In [ ]:
outs_nsw = model_predictions.model_predictions(predDB_nsw, nsw_model, net_size='small_lin')
#outs_nz =  model_predictions.model_predictions(predDB_nz , nz_model)

  0%|          | 0/34 [00:00<?, ?it/s]

filled =223
empty  =49
0.8198529411764706


In [ ]:
outs = pd.concat([outs_nsw,outs_nz])

In [ ]:
outs = outs_nsw

In [ ]:
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.to_pickle(f'output {today}.npy')
outs.to_csv(f'output {state }-{today}.csv')


In [ ]:
pred_df = pd.read_pickle(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\prediction_input 2023-01-26.npy')
pred_df.track_name.value_counts()

Mandurah         95
Shepparton       95
Sandown Park     94
Gunnedah         94
Christchurch     92
Casino           92
Traralgon        90
Warrnambool      89
Albion Park      84
Waikato          83
Hobart           80
Angle Park       77
Dapto            75
Mount Gambier    74
Name: track_name, dtype: int64

In [ ]:
len(pred_df.trackOHE.iloc[0])

71

In [ ]:
x = predDB.racesDict.values()

In [ ]:
torch_saved_dict = torch.load(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\savedmodel\long nsw 2000 RUN\long nsw 2000 RUN_450.pt",map_location=torch.device('cpu'))

In [ ]:
# model_state_dict = torch_saved_dict['model_state_dict']

In [ ]:
predDB.to_cpu()

In [ ]:
predDB.fill_hidden_states_from_dict(hidden_dict=torch_saved_dict['db'])

  0%|          | 0/34 [00:00<?, ?it/s]

filled =218
empty  =54
0.8014705882352942


In [ ]:
predDB.racesDict.values()

dict_values([<rnn_classes.Race object at 0x000001C2B9ABD310>, <rnn_classes.Race object at 0x000001C2B9ABDD00>, <rnn_classes.Race object at 0x000001C2B9ABD6D0>, <rnn_classes.Race object at 0x000001C2B9ABD280>, <rnn_classes.Race object at 0x000001C2B9A80370>, <rnn_classes.Race object at 0x000001C2B9ABD790>, <rnn_classes.Race object at 0x000001C2B9ABDC70>, <rnn_classes.Race object at 0x000001C2B9A806D0>, <rnn_classes.Race object at 0x000001C2B981BF70>, <rnn_classes.Race object at 0x000001C2B9ABD9D0>, <rnn_classes.Race object at 0x000001C2B9ABDE20>, <rnn_classes.Race object at 0x000001C2B9A80490>, <rnn_classes.Race object at 0x000001C2B9A9B4F0>, <rnn_classes.Race object at 0x000001C2B9ABDE50>, <rnn_classes.Race object at 0x000001C2B9ABDE80>, <rnn_classes.Race object at 0x000001C2B9A807C0>, <rnn_classes.Race object at 0x000001C2B9AB8F70>, <rnn_classes.Race object at 0x000001C2B9ABD820>, <rnn_classes.Race object at 0x000001C2B9ABD700>, <rnn_classes.Race object at 0x000001C2B9A80940>, <rnn_cl

In [ ]:
for r in predDB.racesDict.values():
    print(len(r.full_input))

232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232
232


In [ ]:
input_size = predDB.get_race_input([0,1])[0].full_input.shape[0]
model = GRUNet(input_size,64, output='softmax')
model.load_state_dict(torch_saved_dict['model_state_dict'])

<All keys matched successfully>

In [ ]:
model.eval()
outs = model(predDB.racesDict.values()).to('cpu')
predDB.to_cpu()
outs_list = [item for sublist in outs.tolist() for item in sublist]

In [ ]:
dogs = [dog.dog.dog_name for sublist in [r.dogs for r in x] for dog in sublist]
box = [r for s in [[1,2,3,4,5,6,7,8] for r in x] for r in s]
dogId = [dog.dog.dogid for sublist in [r.dogs for r in x] for dog in sublist]
times = [r for s in [[r.race_time]*8 for r in x] for r in s]
rid = [r for s in [[r.raceid]*8 for r in x] for r in s]
track = [r for s in [[r.track_name]*8 for r in x] for r in s]
rnum = [r for s in [[int(r.race_num)]*8 for r in x] for r in s]

In [ ]:
outs = pd.DataFrame(data = {"raceid":rid, "track":track, "racetime":times,"box":box, "dogs":dogs, "dogid":dogId, "conf":outs_list,"race_num":rnum })

In [ ]:
outs['pred_price'] = outs['conf'].apply(lambda x: 1/(x))

In [ ]:
outs = outs.sort_values(['track','race_num','box'])

In [ ]:
outs.head(50)

,raceid,track,racetime,box,dogs,dogid,conf,race_num,pred_price
0,865597968,Casino,10:36AM,1,CUTTIN' GRASS,672317468,0.088841,1,11.256009
1,865597968,Casino,10:36AM,2,CURIOUS CAZA,714918126,0.112474,1,8.890962
2,865597968,Casino,10:36AM,3,SEVEN SINNERS,669960288,0.206793,1,4.835751
3,865597968,Casino,10:36AM,4,SPRITELY GLENNIS,587949066,0.079609,1,12.561409
4,865597968,Casino,10:36AM,5,AMARTEY PARTY,654230428,0.000446,1,2244.363565
5,865597968,Casino,10:36AM,6,CLOVER JULIET,556974388,0.225925,1,4.426243
6,865597968,Casino,10:36AM,7,MR FRESH,714917101,0.117305,1,8.524808
7,865597968,Casino,10:36AM,8,INTEGRITY SON,555068999,0.168607,1,5.930941
8,865597969,Casino,10:56AM,1,ALL THUNDER,592709649,0.168463,2,5.936028
9,865597969,Casino,10:56AM,2,IN THE THIRTIES,471933615,0.000509,2,1964.320351


In [ ]:
outs.track.value_counts()

Casino      96
Gunnedah    96
Dapto       80
Name: track, dtype: int64

In [ ]:
len(pred_df.trackOHE.iloc[0])

71

In [ ]:
today = datetime.datetime.today().strftime('%Y-%m-%d')
outs.to_csv(f'output {state }{today}.csv')

In [ ]:
outs.to_pickle('output 2023-01-24.npy')

In [ ]:
outs.race_num

0       1
1       1
2       1
3       1
4       1
       ..
187    12
188    12
189    12
190    12
191    12
Name: race_num, Length: 272, dtype: int64

In [ ]:
os.getcwd()

'C:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\DATA'

In [ ]:
betfair_df = pd.read_pickle(r'.\betfair races\betfair races 2023-01-24.npy')

In [ ]:
betfair_df

,market_id,track,dist,race_num,runner_id,runnner_name
0,1.209048130,Temora,R2,2,51312709,STRIVE POLDA
1,1.209048130,Temora,R2,2,41385415,TAYLEN BALE
2,1.209048130,Temora,R2,2,51918883,DOUBLE DIPPIN
3,1.209048130,Temora,R2,2,51129843,TAMMY TURBO
4,1.209048130,Temora,R2,2,51918884,QUEST FOR STARS
...,...,...,...,...,...,...
719,1.209048284,Gosford,R6,6,39791347,SOUND OF SILENCE
720,1.209048284,Gosford,R6,6,51918814,WE BACK
721,1.209048284,Gosford,R6,6,39979942,ZIPPING ALABAMA
722,1.209048284,Gosford,R6,6,41962222,NOT A SHAME


In [ ]:
betfair_df.track.value_counts()

Temora           86
Murray Bridge    86
Geelong          86
Ipswich          83
Richmond         80
Launceston       76
Horsham          70
Warragul         45
Grafton          44
Gosford          44
Townsville       24
Name: track, dtype: int64

In [ ]:
outs.race_num = pd.to_numeric(outs.race_num)

In [ ]:
merged = pd.merge(outs, betfair_df, how='left', left_on=['race_num', 'dogs'], right_on=['race_num', 'runnner_name'])

In [ ]:
# betfair_df_9 = betfair_df[betfair_df['race_num']==7]
# outs = outs[outs['race_num']==7]
# merged = pd.merge(outs, betfair_df_9, how='left', left_on=['dogs'], right_on=['runnner_name'])

In [ ]:
merged.track_x.value_counts()

Casino      96
Gunnedah    96
Dapto       80
Name: track_x, dtype: int64

In [ ]:
merged.track_y.value_counts()

Series([], Name: track_y, dtype: int64)

In [ ]:
merged.market_id.unique()

array([nan], dtype=object)